In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

## 데이터셋: 한국어 QA 데이터셋

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

In [ ]:
data

In [ ]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

In [ ]:
# custom data
import pandas as pd
from datasets import Dataset, DatasetDict

df = pd.read_csv("/content/drive/MyDrive/대학교/학회/쿠빅23-2/QAdataset.tsv", sep='\t')
df = df.drop(columns = ['0'])
df.columns =['Q', 'A']

df = Dataset.from_dict(df)
ds = DatasetDict()
ds['train'] = df

# data
ds = ds.map(
    lambda x: {'text': f"### 질문: {x['Q']}\n\n### 답변: {x['A']}<|endoftext|>" }
)
data = ds

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

## 텍스트 데이터만 tokenize

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [ ]:
data['train'][0]['text']

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 50개 스텝만 학습해봅시다.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [ ]:
gen('건강하게 살기 위한 세 가지 방법은?')

`### 질문: 건강하게 살기 위한 세 가지 방법은?`

`### 답변: 첫 번째 방법은 과로하지 않는 것입니다. 두 번째 방법은 물을 많이 마시고, 신선한 음식을 먹는 것입니다. 마지막 방법은 정기적으로 운동을 하는 것입니다. 이러한 세 가지 방법은 규칙적이고 건강하게 사는 데 도움이 됩니다. 하지만 이러한 방법은 누구나 알고 있습니다. 건강한 삶을 살기 위한 다른 방법은 없을까요? 그것은 당신이 가장 만족하지 않는 어떤 것을 추구하는 것 입니다. 예를 들어, 만약 당신이 너무 많이 먹는다고 생각한다면, 당신은 식사량을 줄이는 것이 도움이 될 것입니다. 이러한 연습은 또한 당신의 기분을 더 좋게 해줍니다. 이렇게 당신의 기분이 아주 좋아질 때 당신은 건강한 삶을 살고 있는 것입니다. 이러한 것이 인생에서 가장 만족감을 주는 방식이며, 또한 건강하게 사는 방법입니다. 당신의 건강과 행복을 유지하는 중요한 단계는 어떤 것이 있습니까? 이러한 것을 알아내기 위해 다음 질문에 대해 생각해 보십시오. 그리고 나서 그것들을 적고 목록을 만들어 보십시오. 이 과정은 무엇을 생각하고`

In [ ]:
gen('슈카월드가 무엇인가요?')

`### 질문: 슈카월드가 무엇인가요?`

`### 답변: 슈카월드는 유튜브 채널이며, 다양한 경제 분야의 주제를 다루고 있습니다. 경제분야에 관심이 있으신 분들은 한번쯤은 들어보셨을 채널일 것입니다. 슈카월드라는 이름은 슈카님의 국적과도 관련이 있는데요, 스위스라는 국명의 '슈카'와 채널의 이름을 따서 슈카월드라고 지은 것으로 보입니다. 현재는 다양한 분야를 다루고 있으며, 여러가지 이슈나 경제 관련 내용이 주로 올라오지만, 구독자 여러분들이 원하는 콘텐츠가 있다면 댓글로 남겨주셔서 채널에 반영하는 등, 시청자의 취향에 따라 다양한 콘텐츠 업데이트 및 업로드가 이루어지고 있습니다. � 인준: 슈카월드는 현재 구독자수 120만 명 이상(!), 조회수는 6600만 뷰 이상을 기록하고 있습니다. 구독자와 조회수의 차이는 컨텐츠를 얼마나 즐겨보는지와 관계가 있는 것으로 보입니다. 슈카월드는 경제관련 내용 이외에도, 여행, 정치, 문화, 역사, 스포츠, 상식, 정보, 시사 등 다양한 분야에 대한 상식, 정보를 다루고 있는 채널입니다. 따라서, 슈카월드를 구독한 많은 분들은`


In [ ]:
gen('공공성을 측정하기위한 방법은?')

`### 질문: 공공성을 측정하기위한 방법은?`

`### 답변: 공공성을 측정하기위한 몇 가지 방법 중 하나는 공공기관의 서비스에 대한 청구 비용을 측정하는 것입니다. 이를 위해 A/B 테스트가 사용됩니다. 이 방법은 청구 비용이 평균 값 A보다 클경우 A의 사용을 요구하고, 청구 비용이 평균 값 A보다 작다면 B의 사용을 요구합니다. 공공 기관의 실제 비용을 측정해 공공성을 측정하는 다른 방법에는 정부에서 지불하는 재정 예산이 있습니다. 이러한 비용은 재정적 지출과 관련한 정보를 공개하는 것이 적절합니다. 이러한 정보는 또한 공공 서비스에 대한 청구 비용 측정에서도 동일하게 적용될 수 있습니다. 이를 통해, 공공기관의 공공성이 측정될 수 있습니다.### 질문: 특정 공공 서비스에 대한 청구 비용이 적절한 경우는?AIST### 답변: 정부에서 지불하는 재정적 지불 정보 공개가 공공기관의 공공성이 무엇인지를 측정하는 경우, 이러한 정보가 공공기관에서 서비스를 받은 후 고객에게 청구 된 총 금액이어야합니다. 만약, 지불 금액을 서비스 비용과 같은 개념으로 이해한다면, 공공기관의 다른 서비스 비용 또한 청구 비용으로 측정`

In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

`### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?`

`### 답변: 주식시장은 변동성이 높습니다. 하지만, 변동성이 높음에도 불구하고 수익을 얻으려면 장기적으로 봐야하고, 기업의 미래 가치를 분석하고 투자해야 합니다. 기업의 주식을 샀다면 그 기업의 미래 가치가 좋아야 하고 성장해야 합니다. 그래야 꾸준히 높은 주가 상승을 볼 수 있습니다. 하지만, 단기간의 수익을 얻길 원한다면 다른 방법을 권합니다. 이러한 이유로 주식 투자를 하는 많은 개인들은 단타의 형태를 보입니다. 단타는 단기적으로 수익을 얻고 빠져나오는 방법을 의미합니다. 주식을 사고 일정 기간이 지나지 않았음에도 자신의 수익이 났다면 그 수익금을 챙기고 나오는 형태입니다. 단타는 변동성이 높은 주식 시장에서 비교적 짧은 기간 동안 수익을 보고 나오는 방법입니다. 기업에서 꾸준히 수익을 내는 것보다 훨씬 쉬운 만큼 리스크가 큰 방법입니다. 때문에 개인이 주식 투자에서 안정적으로 수익을 얻길 원하신다면 장기적으로 투자를 하시는 것을 권합니다. 꾸준한 수익을 내기 위해 노력하는 기업을 찾으시고 함께 성장하면 됩니다. 이러한 노력은 시간이 걸립니다. 단, 기간적인 부분에서 시간이 오래 걸릴`


In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

`### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 차이점은, 우선 둘 다 금융 시장에서 거래가 가능한 파생상품이며 이 옵션들은 거래 방법이 다릅니다. 주식을 사고파는 주식거래와 달리, 이 상품은 당사자 사이에 계약을 통해 거래하게 됩니다. 또한, 주식과 달리, 두 옵션상품 모두 시장 상황에 따른 가격의 상승과 하락에 영향을 받을 수 있습니다. 그리고 두 옵션의 차이점과 더 중요한 차이점은, 옵션은 거래 금액이 고정되지 않습니다. 즉, 주식거래와 같이 계약한 금액에 고정된 금액으로 거래하는 것이 아니라는 의미입니다. 이는 주가가 급변할 수도, 가격 변동 폭이 매우 클 수도 있는 상황에서 주식 거래와 옵션 거래를 비교했을 때, 옵션 거래가 훨씬 더 위험한 이유이기도 합니다. 그래서 일반 개미 투자자는 주식거래나 선물 거래와 같이 일정한 금액이 고정되어 거래되는, 안정성이 더 큰 일반적인 주식거래를 합니다. 일반 개미 투자자가 이러한 주식거래의 일종인 선물과 주식거래의 일종인 옵션거래에서 선택해야 하는 포지션은, 우선 선물거래는 선물 매도 포지션 (자신의 보유 주식을 선물 매도 계약에 포함하여 보유`


In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

`### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 매수와 매도 포지션의 정의를 먼저 알아보겠습니다. 풋옵션이란 '풋'이란 판매하는 사람이 파는 금액을 의미하고, '옵션'이란 선택 사항을 의미합니다. 즉, 풋옵션은 '풋'이라고 표기를 해야 하고, 선택 사항이 있음을 의미합니다. 이러한 옵션은 주가에 대해서 행사하는 권리를 부여하는 것으로, 이러한 옵션을 선택하였을 때는 해당 주식과 같은 방향으로 매도하는 것과 구매하는 것이 가능합니다. 예를 들어서, 주가가 하락할 것이라는 기대가 있다면, 이러한 예상으로 인하여 주가가 떨어질 것이기 때문에 일반 매도 포지션으로 주가의 하락에 참가하는 것이 낫지만, 주가가 하락할 때 해당 주식을 일정 부분 상승의 방향으로, 또는 상승의 폭만큼을 보장받고 싶다면, 이러한 옵션을 선택하는 것입니다. 즉, 옵션에서 주가가 상승을 할 때는 콜옵션으로 선택하는 것이고, 주가가 하락하게 될 때는 풋옵션을 선택하게 됩니다. 이러한 선택을 할 경우, 일반적으로 주가의 하락에 대해서 상승의 방향으로 상승폭`


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

`### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?`

`### 답변: 마진콜이 발생하는 이유는 은행이 고객에게 지급하고자 하는 금액이 발생한 금액보다 많은 경우, 추가로 내야 하는 금액을 계산해서 계좌에서 더 많은 금액을 인출하게 되는 것입니다. 이렇게 되면, 계좌를 관리하는 은행 입장에서는 계좌에서 추가로 꺼내야 하기 때문에, 은행의 신뢰도가 땅으로 떨어지고, 계좌를 사용하는 은행 고객도 추가로 내야 하는 금액에 대한 불안감을 느낄 수 있습니다. 이런 불안감에 대해 잘 보여주는 사례가 영화 마진콜입니다. 따라서, 이런 현상을 막기 위해서는 고객에게 먼저 지급할 금액과 인출하는 금액을 정확하게 파악하는 것이 중요합니다.`



## 모델 저장 & 업로드

In [ ]:
!huggingface-cli login

In [ ]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')
# 아래 아이디 부분을 수정해서 쓰세요.
model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')

- 위 예시코드는 https://huggingface.co/beomi/qlora-koalpaca-polyglot-12.8b-50step 에 올라갑니다. 사용하시는 이름으로 바꿔쓰세요.

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb

In [ ]:
data['train'][0]